# General

The job of this notebook is to:

* Load the preprocessed tracking data into a `master_df` DataFrame

* Add handcrafted features as columns to `master_df`

* Save the `master_df` so it can be used for later classification notebooks

## Style / naming guide / coding hints

* Axes in column labels such as 'X' and 'Y' are always capitalized: e.g. `left_ear_X`
* All coordinates are in cm, area in cm^2
* All angles are in degrees
* 0° is up / north in the video frame
* whenever extracting features, do sanity checks on them:
  * plot time course: are there weird discontinuities etc.?
* often, rounding values makes sense to reduce numerical errors (e.g. angles such as 34.999999° are biologically meaningless -> round to 35°

# Housekeeping

In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import colors
from matplotlib import rc
import matplotlib.patches as mpatches

# Loading the data from Google Drive

In [2]:
#Because we are using a Shared with me drive, we need to have a small workaround
#In your shared with me folder, right click on:
#NMA_2022_Sfenj_Group_1"
#And click "Make a shortcut", and have this shortcut within your own drive

#now we can mount our drive
from pickle import load
# mount google drive
from google.colab import drive
drive.mount('/content/drive')
#Since you have mounted your drive, which includes this shortcut, you can now access the files within that shortcut:
master_df = pd.read_csv('/content/drive/MyDrive/NMA_2022_Sfenj_Group_1/master_df.csv')

#test by showing the head
master_df.head()

Mounted at /content/drive


,Unnamed: 0,frame_num,session_num,mouse_id,annotations,nose_X,nose_Y,left_ear_X,left_ear_Y,right_ear_X,right_ear_Y,neck_X,neck_Y,left_hip_X,left_hip_Y,right_hip_X,right_hip_Y,tailbase_X,tailbase_Y
0,0,0,0,0,3,831.659204,202.914433,805.659204,250.914433,775.659204,189.914433,780.659204,225.914433,711.659204,278.914433,711.659204,192.914433,643.659204,220.914433
1,1,1,0,0,1,833.050439,201.895063,809.050439,251.895063,778.050439,193.895063,783.050439,229.895063,723.050439,287.895063,717.050439,192.895063,644.050439,227.895063
2,2,2,0,0,1,838.718976,179.862692,816.718976,244.862692,776.718976,193.862692,787.718976,225.862692,730.718976,286.862692,713.718976,196.862692,646.718976,233.862692
3,3,3,0,0,1,826.757507,175.148063,815.757507,235.148063,774.757507,187.148063,785.757507,218.148063,743.757507,282.148063,711.757507,198.148063,644.757507,237.148063
4,4,4,0,0,1,822.045709,174.457936,812.045709,222.457936,768.045709,178.457936,779.045709,211.457936,749.045709,278.457936,709.045709,194.457936,646.045709,233.457936


## Conversion to cm 
The tracking data is provided in "pixel" units, to convert into Centimeters units, all coordinates need to be divided by 37.7

In [3]:
## converts dataframe into units of cm
## uses conversion of 37.7 pixels/cm
x = master_df.columns.tolist()
x= x[5:]
master_df[x]= master_df[x]/37.7

# Adding features

#### Postion centroids

Centroids are calculated by taking the mean of the individual coordinates.

In [4]:
#Now we can calculate features in the master_df
#re-create indexing variables from when master_df was initially created
master_dict = {'frame_num':[], 'session_num': [], 'mouse_id': [], 'annotations':[]}
pos_x_names = ['nose_X', 'left_ear_X', 'right_ear_X','neck_X', 'left_hip_X', 'right_hip_X', 'tailbase_X'] 
pos_y_names = ['nose_Y', 'left_ear_Y', 'right_ear_Y','neck_Y', 'left_hip_Y', 'right_hip_Y', 'tailbase_Y']

# get average of all XY positions for centroid location
#calcuate the mean of all x's and y's (centroid of all 7 pts)
master_df['centroid_X'] = master_df[pos_x_names].mean(axis=1) 
master_df['centroid_Y'] = master_df[pos_y_names].mean(axis=1)

master_df['centroid_head_X'] = master_df[['nose_X', 'left_ear_X', 'right_ear_X','neck_X']].mean(axis=1)
master_df['centroid_head_Y'] = master_df[['nose_Y', 'left_ear_Y', 'right_ear_Y','neck_Y']].mean(axis=1)
master_df['centroid_body_X'] = master_df[['left_hip_X', 'right_hip_X', 'tailbase_X']].mean(axis=1)
master_df['centroid_body_Y'] = master_df[['left_hip_Y', 'right_hip_Y', 'tailbase_Y']].mean(axis=1)

master_df.head()

,Unnamed: 0,frame_num,session_num,mouse_id,annotations,nose_X,nose_Y,left_ear_X,left_ear_Y,right_ear_X,...,right_hip_X,right_hip_Y,tailbase_X,tailbase_Y,centroid_X,centroid_Y,centroid_head_X,centroid_head_Y,centroid_body_X,centroid_body_Y
0,0,0,0,0,3,22.059926,5.382346,21.370271,6.655555,20.574515,...,18.876902,5.117094,17.073188,5.859799,19.934121,5.920428,21.177963,5.766961,18.275664,6.125051
1,1,1,0,0,1,22.096829,5.355307,21.460224,6.681567,20.637943,...,19.019906,5.116580,17.083566,6.044962,20.035442,6.010858,21.241391,5.819498,18.427509,6.266005
2,2,2,0,0,1,22.247188,4.770894,21.663633,6.495032,20.602625,...,18.931538,5.221822,17.154349,6.203254,20.125172,5.919056,21.351962,5.599806,18.489451,6.344722
3,3,3,0,0,1,21.929907,4.645837,21.638130,6.237349,20.550597,...,18.879509,5.255917,17.102321,6.290400,20.095879,5.809157,21.240252,5.408437,18.570049,6.343450
4,4,4,0,0,1,21.804926,4.627531,21.539674,5.900741,20.372565,...,18.807578,5.158035,17.136491,6.192518,20.027738,5.658225,21.095377,5.217717,18.604219,6.245569


#### Head Angles

In [7]:
vec_tail2neck_x = master_df['neck_X'] - master_df['tailbase_X']
vec_tail2neck_y = master_df['neck_Y'] - master_df['tailbase_Y']
vec_neck2nose_x = master_df['nose_X'] - master_df['neck_X']
vec_neck2nose_y = master_df['nose_Y'] - master_df['neck_Y']

v_ = vec_tail2neck_x * vec_neck2nose_y - vec_tail2neck_y * vec_neck2nose_x
w_ = vec_tail2neck_x * vec_neck2nose_x + vec_tail2neck_y * vec_neck2nose_y

master_df['head_vs_body_angle'] = np.degrees(np.arctan2(v_,w_))

#### Body angle calculation

In [8]:
vec_neck2nose_x = master_df['nose_X'] - master_df['neck_X']
vec_neck2nose_y = master_df['nose_Y'] - master_df['neck_Y']
# basis vector pointing north
vec_x = 0
vec_y = 1

v_ = vec_x * vec_neck2nose_y - vec_y * vec_neck2nose_x
w_ = vec_x * vec_neck2nose_x + vec_y * vec_neck2nose_y

master_df['head_orientation'] = np.degrees(np.arctan2(v_,w_))

## Unstable / Playground below (add your code here)